In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

spam.csv



In [2]:
DATA_FILE = '../input/spam.csv'
df = pd.read_csv(DATA_FILE,encoding='latin-1')
print(df.head())

tags = df.v1
texts = df.v2
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Embedding
from keras.layers import Conv1D, GlobalMaxPooling1D
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from sklearn.preprocessing import LabelEncoder
import time
from keras import metrics
print('import done')

     v1                                                 v2 Unnamed: 2  \
0   ham  Go until jurong point, crazy.. Available only ...        NaN   
1   ham                      Ok lar... Joking wif u oni...        NaN   
2  spam  Free entry in 2 a wkly comp to win FA Cup fina...        NaN   
3   ham  U dun say so early hor... U c already then say...        NaN   
4   ham  Nah I don't think he goes to usf, he lives aro...        NaN   

  Unnamed: 3 Unnamed: 4  
0        NaN        NaN  
1        NaN        NaN  
2        NaN        NaN  
3        NaN        NaN  
4        NaN        NaN  


Using TensorFlow backend.


import done


In [3]:
num_max = 1000
# preprocess
le = LabelEncoder()
tags = le.fit_transform(tags)
tok = Tokenizer(num_words=num_max)
tok.fit_on_texts(texts)
mat_texts = tok.texts_to_matrix(texts,mode='count')
print(tags[:5])
print(mat_texts[:5])
print(tags.shape,mat_texts.shape)

[0 0 1 0 0]
[[ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  3. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  1.  1. ...,  0.  0.  0.]]
(5572,) (5572, 1000)


In [4]:
# try a simple model first

def get_simple_model():
    model = Sequential()
    model.add(Dense(512, activation='relu', input_shape=(num_max,)))
    model.add(Dropout(0.2))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(1, activation='sigmoid'))
    model.summary()
    model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['acc',metrics.binary_accuracy])
    print('compile done')
    return model

def check_model(model,x,y):
    model.fit(x,y,batch_size=32,epochs=10,verbose=1,validation_split=0.2)

m = get_simple_model()
check_model(m,mat_texts,tags)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 512)               512512    
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 256)               131328    
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 257       
Total params: 644,097
Trainable params: 644,097
Non-trainable params: 0
_________________________________________________________________
compile done


/opt/conda/lib/python3.6/site-packages/Keras-2.0.4-py3.6.egg/keras/backend/tensorflow_backend.py:2252: UserWarning: Expected no kwargs, you passed 1
kwargs passed to function are ignored with Tensorflow backend
  warnings.warn('\n'.join(msg))


Train on 4457 samples, validate on 1115 samples
Epoch 1/10
4457/4457 [==============================] - 4s - loss: 0.1575 - acc: 0.9506 - binary_accuracy: 0.9506 - val_loss: 0.0600 - val_acc: 0.9883 - val_binary_accuracy: 0.9883
Epoch 2/10
4457/4457 [==============================] - 3s - loss: 0.0341 - acc: 0.9912 - binary_accuracy: 0.9912 - val_loss: 0.0735 - val_acc: 0.9865 - val_binary_accuracy: 0.9865
Epoch 3/10
4457/4457 [==============================] - 3s - loss: 0.0096 - acc: 0.9980 - binary_accuracy: 0.9980 - val_loss: 0.0722 - val_acc: 0.9857 - val_binary_accuracy: 0.9857
Epoch 4/10
4457/4457 [==============================] - 3s - loss: 0.0041 - acc: 0.9989 - binary_accuracy: 0.9989 - val_loss: 0.0903 - val_acc: 0.9857 - val_binary_accuracy: 0.9857
Epoch 5/10
4457/4457 [==============================] - 3s - loss: 0.0023 - acc: 0.9996 - binary_accuracy: 0.9996 - val_loss: 0.0946 - val_acc: 0.9865 - val_binary_accuracy: 0.9865
Epoch 6/10
4457/4457 [=========================

In [5]:
# for cnn preproces
max_len = 100
cnn_texts_seq = tok.texts_to_sequences(texts)
print(cnn_texts_seq[0])
cnn_texts_mat = sequence.pad_sequences(cnn_texts_seq,maxlen=max_len)
print(cnn_texts_mat[0])
print(cnn_texts_mat.shape)

[50, 469, 841, 751, 657, 64, 8, 89, 121, 349, 147, 67, 58, 144]
[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0  50 469 841 751
 657  64   8  89 121 349 147  67  58 144]
(5572, 100)


In [6]:
def get_cnn_model_v1():   
    model = Sequential()
    # we start off with an efficient embedding layer which maps
    # our vocab indices into embedding_dims dimensions
    # 1000 is num_max
    model.add(Embedding(1000,
                        20,
                        input_length=max_len))
    model.add(Dropout(0.2))
    model.add(Conv1D(64,
                     3,
                     padding='valid',
                     activation='relu',
                     strides=1))
    model.add(GlobalMaxPooling1D())
    model.add(Dense(256))
    model.add(Dropout(0.2))
    model.add(Activation('relu'))
    model.add(Dense(1))
    model.add(Activation('sigmoid'))
    model.summary()
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['acc',metrics.binary_accuracy])
    return model

m = get_cnn_model_v1()
check_model(m,cnn_texts_mat,tags)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 100, 20)           20000     
_________________________________________________________________
dropout_3 (Dropout)          (None, 100, 20)           0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 98, 64)            3904      
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 64)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 256)               16640     
_________________________________________________________________
dropout_4 (Dropout)          (None, 256)               0         
_________________________________________________________________
activation_1 (Activation)    (None, 256)               0         
__________

/opt/conda/lib/python3.6/site-packages/Keras-2.0.4-py3.6.egg/keras/backend/tensorflow_backend.py:2252: UserWarning: Expected no kwargs, you passed 1
kwargs passed to function are ignored with Tensorflow backend
  warnings.warn('\n'.join(msg))


Train on 4457 samples, validate on 1115 samples
Epoch 1/10
4457/4457 [==============================] - 4s - loss: 0.3981 - acc: 0.8649 - binary_accuracy: 0.8649 - val_loss: 0.2687 - val_acc: 0.8700 - val_binary_accuracy: 0.8700
Epoch 2/10
4457/4457 [==============================] - 4s - loss: 0.1740 - acc: 0.9336 - binary_accuracy: 0.9336 - val_loss: 0.0694 - val_acc: 0.9785 - val_binary_accuracy: 0.9785
Epoch 3/10
4457/4457 [==============================] - 4s - loss: 0.0508 - acc: 0.9863 - binary_accuracy: 0.9863 - val_loss: 0.0496 - val_acc: 0.9821 - val_binary_accuracy: 0.9821
Epoch 4/10
4457/4457 [==============================] - 4s - loss: 0.0302 - acc: 0.9933 - binary_accuracy: 0.9933 - val_loss: 0.0536 - val_acc: 0.9839 - val_binary_accuracy: 0.9839
Epoch 5/10
4457/4457 [==============================] - 4s - loss: 0.0227 - acc: 0.9942 - binary_accuracy: 0.9942 - val_loss: 0.0575 - val_acc: 0.9857 - val_binary_accuracy: 0.9857
Epoch 6/10
4457/4457 [=========================

In [7]:
def get_cnn_model_v2(): # added embed   
    model = Sequential()
    # we start off with an efficient embedding layer which maps
    # our vocab indices into embedding_dims dimensions
    # 1000 is num_max
    model.add(Embedding(1000,
                        50, #!!!!!!!!!!!!!!!!!!!!!!!
                        input_length=max_len))
    model.add(Dropout(0.2))
    model.add(Conv1D(64,
                     3,
                     padding='valid',
                     activation='relu',
                     strides=1))
    model.add(GlobalMaxPooling1D())
    model.add(Dense(256))
    model.add(Dropout(0.2))
    model.add(Activation('relu'))
    model.add(Dense(1))
    model.add(Activation('sigmoid'))
    model.summary()
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['acc',metrics.binary_accuracy])
    return model

m = get_cnn_model_v2()
check_model(m,cnn_texts_mat,tags)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 100, 50)           50000     
_________________________________________________________________
dropout_5 (Dropout)          (None, 100, 50)           0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 98, 64)            9664      
_________________________________________________________________
global_max_pooling1d_2 (Glob (None, 64)                0         
_________________________________________________________________
dense_6 (Dense)              (None, 256)               16640     
_________________________________________________________________
dropout_6 (Dropout)          (None, 256)               0         
_________________________________________________________________
activation_3 (Activation)    (None, 256)               0         
__________

/opt/conda/lib/python3.6/site-packages/Keras-2.0.4-py3.6.egg/keras/backend/tensorflow_backend.py:2252: UserWarning: Expected no kwargs, you passed 1
kwargs passed to function are ignored with Tensorflow backend
  warnings.warn('\n'.join(msg))


Train on 4457 samples, validate on 1115 samples
Epoch 1/10
4457/4457 [==============================] - 6s - loss: 0.3709 - acc: 0.8613 - binary_accuracy: 0.8613 - val_loss: 0.2361 - val_acc: 0.8700 - val_binary_accuracy: 0.8700
Epoch 2/10
4457/4457 [==============================] - 6s - loss: 0.1858 - acc: 0.9217 - binary_accuracy: 0.9217 - val_loss: 0.0657 - val_acc: 0.9830 - val_binary_accuracy: 0.9830
Epoch 3/10
4457/4457 [==============================] - 6s - loss: 0.0493 - acc: 0.9872 - binary_accuracy: 0.9872 - val_loss: 0.0481 - val_acc: 0.9830 - val_binary_accuracy: 0.9830
Epoch 4/10
4457/4457 [==============================] - 6s - loss: 0.0282 - acc: 0.9930 - binary_accuracy: 0.9930 - val_loss: 0.0524 - val_acc: 0.9874 - val_binary_accuracy: 0.9874
Epoch 5/10
4457/4457 [==============================] - 6s - loss: 0.0185 - acc: 0.9955 - binary_accuracy: 0.9955 - val_loss: 0.0552 - val_acc: 0.9857 - val_binary_accuracy: 0.9857
Epoch 6/10
4457/4457 [=========================

In [8]:
def get_cnn_model_v3():    # added filter
    model = Sequential()
    # we start off with an efficient embedding layer which maps
    # our vocab indices into embedding_dims dimensions
    # 1000 is num_max
    model.add(Embedding(1000,
                        20,
                        input_length=max_len))
    model.add(Dropout(0.2))
    model.add(Conv1D(256, #!!!!!!!!!!!!!!!!!!!
                     3,
                     padding='valid',
                     activation='relu',
                     strides=1))
    model.add(GlobalMaxPooling1D())
    model.add(Dense(256))
    model.add(Dropout(0.2))
    model.add(Activation('relu'))
    model.add(Dense(1))
    model.add(Activation('sigmoid'))
    model.summary()
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['acc',metrics.binary_accuracy])
    return model

m = get_cnn_model_v3()
check_model(m,cnn_texts_mat,tags)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 100, 20)           20000     
_________________________________________________________________
dropout_7 (Dropout)          (None, 100, 20)           0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 98, 256)           15616     
_________________________________________________________________
global_max_pooling1d_3 (Glob (None, 256)               0         
_________________________________________________________________
dense_8 (Dense)              (None, 256)               65792     
_________________________________________________________________
dropout_8 (Dropout)          (None, 256)               0         
_________________________________________________________________
activation_5 (Activation)    (None, 256)               0         
__________

/opt/conda/lib/python3.6/site-packages/Keras-2.0.4-py3.6.egg/keras/backend/tensorflow_backend.py:2252: UserWarning: Expected no kwargs, you passed 1
kwargs passed to function are ignored with Tensorflow backend
  warnings.warn('\n'.join(msg))


Train on 4457 samples, validate on 1115 samples
Epoch 1/10
4457/4457 [==============================] - 15s - loss: 0.3615 - acc: 0.8638 - binary_accuracy: 0.8638 - val_loss: 0.2385 - val_acc: 0.8700 - val_binary_accuracy: 0.8700
Epoch 2/10
4457/4457 [==============================] - 15s - loss: 0.2024 - acc: 0.9165 - binary_accuracy: 0.9165 - val_loss: 0.1460 - val_acc: 0.9426 - val_binary_accuracy: 0.9426
Epoch 3/10
4457/4457 [==============================] - 16s - loss: 0.0643 - acc: 0.9832 - binary_accuracy: 0.9832 - val_loss: 0.0425 - val_acc: 0.9857 - val_binary_accuracy: 0.9857
Epoch 4/10
4457/4457 [==============================] - 15s - loss: 0.0330 - acc: 0.9908 - binary_accuracy: 0.9908 - val_loss: 0.0518 - val_acc: 0.9830 - val_binary_accuracy: 0.9830
Epoch 5/10
4457/4457 [==============================] - 16s - loss: 0.0233 - acc: 0.9942 - binary_accuracy: 0.9942 - val_loss: 0.0478 - val_acc: 0.9857 - val_binary_accuracy: 0.9857
Epoch 6/10
4457/4457 [====================